In [0]:
#install dependencies
%pip install ipywidgets
%pip install nbformat>=4.2.0

In [0]:
dbutils.library.restartPython()

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from pyspark.sql import SparkSession

#Start Spark Session
spark = SparkSession.builder.getOrCreate()
df_spark = spark.table("workspace.default.stock_metrics")
df = df_spark.toPandas()
df['DATETIME'] = pd.to_datetime(df['DATETIME'])

#Widgets Selectors
available_tickers = df['TICKER'].unique().tolist()
ticker_select = widgets.SelectMultiple(
    options=available_tickers,
    value=[available_tickers[0]],
    description='Tickers'
)

date_range = widgets.SelectionRangeSlider(
    options=[(d.strftime("%Y-%m-%d"), d) for d in pd.date_range(df['DATETIME'].min(), df['DATETIME'].max())],
    index=(0, len(pd.date_range(df['DATETIME'].min(), df['DATETIME'].max()))-1),
    description='Date Range',
    orientation='horizontal',
    layout={'width': '800px'},
    readout=True
)

ma_15_chk = widgets.Checkbox(value=True, description='MA15')
ma_30_chk = widgets.Checkbox(value=False, description='MA30')
ma_60_chk = widgets.Checkbox(value=False, description='MA60')
bb_chk = widgets.Checkbox(value=True, description='Bollinger Bands')

ma_box = widgets.HBox([ma_15_chk, ma_30_chk, ma_60_chk, bb_chk])
controls = widgets.VBox([ticker_select, date_range, ma_box])
display(controls)

plot_out = widgets.Output()
display(plot_out)

#Colours
line_colors = {
    'Close': 'black',
    'MA15': 'orange',
    'MA30': 'green',
    'MA60': 'purple',
    'BB15': 'black',
    'BB30': 'lightgreen',
    'BB60': 'violet'
}

def update_charts(change=None):
    start_date, end_date = date_range.value
    selected_tickers = ticker_select.value
    MA15, MA30, MA60, BB = ma_15_chk.value, ma_30_chk.value, ma_60_chk.value, bb_chk.value

    df_filtered = df[(df['DATETIME'] >= start_date) & (df['DATETIME'] <= end_date)]

    with plot_out:
        plot_out.clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(10,6))

        for ticker in selected_tickers:
            df_t = df_filtered[df_filtered['TICKER'] == ticker]
            ax.plot(df_t['DATETIME'], df_t['CLOSE'], label=f"{ticker} Close", color=line_colors['Close'])
            
            if MA15:
                ax.plot(df_t['DATETIME'], df_t['MA_15'], label=f"{ticker} MA15", color=line_colors['MA15'])
                if BB:
                    label = "BB15" if 'BB15' not in ax.get_legend_handles_labels()[1] else None
                    ax.fill_between(df_t['DATETIME'], df_t['MA_15_LOWER_BB'], df_t['MA_15_UPPER_BB'],
                                    alpha=0.2, facecolor=line_colors['BB15'], edgecolor=line_colors['BB15'],
                                    label=label)
            if MA30:
                ax.plot(df_t['DATETIME'], df_t['MA_30'], label=f"{ticker} MA30", color=line_colors['MA30'])
                if BB:
                    label = "BB30" if 'BB30' not in ax.get_legend_handles_labels()[1] else None
                    ax.fill_between(df_t['DATETIME'], df_t['MA_30_LOWER_BB'], df_t['MA_30_UPPER_BB'],
                                    alpha=0.2, facecolor=line_colors['BB30'], edgecolor=line_colors['BB30'],
                                    label=label)
            if MA60:
                ax.plot(df_t['DATETIME'], df_t['MA_60'], label=f"{ticker} MA60", color=line_colors['MA60'])
                if BB:
                    label = "BB60" if 'BB60' not in ax.get_legend_handles_labels()[1] else None
                    ax.fill_between(df_t['DATETIME'], df_t['MA_60_LOWER_BB'], df_t['MA_60_UPPER_BB'],
                                    alpha=0.2, facecolor=line_colors['BB60'], edgecolor=line_colors['BB60'],
                                    label=label)
        
        ax.set_title("Price, Moving Averages & Bollinger Bands")
        ax.set_xlabel("Date")
        ax.set_ylabel("Price")
        ax.legend(loc='upper left', bbox_to_anchor=(1.0, 1.0))
        plt.show()

#Observer changes to ticker
ticker_select.observe(update_charts, names='value')
date_range.observe(update_charts, names='value')
ma_15_chk.observe(update_charts, names='value')
ma_30_chk.observe(update_charts, names='value')
ma_60_chk.observe(update_charts, names='value')
bb_chk.observe(update_charts, names='value')

#Initialize Plot
update_charts()
